In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
#import gymnasium as gym
import numpy as np
from envs.two_player_dubins_car import TwoPlayerDubinsCarEnv


import jax
import jax.numpy as jnp
import haiku as hk
import optax

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import pickle 


In [2]:
def policy_network(observation):
    net = hk.Sequential([
        hk.Linear(100), jax.nn.relu,
        hk.Linear(100), jax.nn.relu,
        hk.Linear(100), jax.nn.relu,
        hk.Linear(100), jax.nn.relu,

        hk.Linear(env.num_actions),
        jax.nn.softmax
    ])
    return net(observation)

policy_net = hk.without_apply_rng(hk.transform(policy_network))
key = jax.random.PRNGKey(42)
epsilon = 0.1

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
#Load data (deserialize)
with open('data/drone/test.pickle', 'rb') as handle:
     loaded_params = pickle.load(handle)

env = TwoPlayerDubinsCarEnv()
env.reset()
state = env.state
state

/home/arjun/.local/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


{'attacker': array([2., 2., 0.], dtype=float32),
 'defender': array([0., 0., 0.], dtype=float32)}

0
0


In [5]:
#


timesteps = np.arange(0, 10, 0.1)
for t in timesteps:
    if t % 0.5 == 0:
        prev_state = state
        for player in env.players:
            key, subkey = jax.random.split(key)
            nn_state = env.encode_helper(state)
            action = env.select_action_sr(nn_state, loaded_params[player], policy_net,  subkey, epsilon)
            state, reward, done, info = env.step(state=state, action=action, player=player, update_env=True)
            print(action)
        pass

    attacker_prev_state = prev_state['attacker'][0:3]
    defender_prev_state = prev_state['defender'][0:3]

    attacker_state = state['attacker'][0:3]
    defender_state = state['defender'][0:3]

    lamb = (t % 0.5) * 2 
    attacker_pos = lamb * attacker_state + (1 - lamb) * attacker_prev_state  
    defender_pos = lamb * defender_state + (1 - lamb) * defender_prev_state


    # send position to crazyflie
    # wait for 0.1 seconds

1
0
0
1
0
1
2
0
2
2
0
0
1
0
1
2
0
0
0
1
2
1
0
1
0
0
0
2
2
2
1
1
1
0
2
0
0
0
0
1
